In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

# Parte 1

Se usaran los datos del dataset Titanic.

3 modelos de clasificador:

- regresión logística
- K-NN
- XGBoost

Con diferentes hiperparámetros con el mismo conjunto de datos.

métricas: 

- la precisión
- la puntuación F1

In [ ]:
import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://Minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

In [ ]:
df = sns.load_dataset('titanic')
df = df[['survived', 'pclass', 'age', 'sibsp', 'parch', 'adult_male']].dropna().reset_index(drop = True)
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('survived', axis = 1), df['survived'], test_size=0.2, random_state=42)

Configuremos los experimentos para que MLFlow realice un seguimiento. Primero, necesitamos configurar el URI de seguimiento, que apunta a nuestro servidor local (docker).

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://tester-mlflow-1:5000")

Configuramos el grupo Experimento

In [ ]:
EXPERIMENT_NAME = "Titanic-Survived-Classifier-Experiment"
mlflow.set_experiment(EXPERIMENT_NAME)

No necesitamos crearlo nuevamente si desea volver a este experimento. En cambio, solo necesitamos el identificador de ID del experimento para obtener el experimento deseado y pasar el ID durante el proceso de seguimiento.

In [ ]:
current_experiment=dict(mlflow.get_experiment_by_name(EXPERIMENT_NAME))
experiment_id=current_experiment['experiment_id']

Para el experimento, lo configuraríamos con la configuración actual.

In [ ]:
models_to_experiment = {
        "LR": LogisticRegression,
        "K-NN": KNeighborsClassifier,
        "XGB Classifier": XGBClassifier
    }

params_to_experiment = {
    'LR': {
        'solver': 'lbfgs',
        'C': 1,
        'max_iter': 100
    },
    'K-NN': {
        'n_neighbors': 5,
        'weights': 'uniform',
        'algorithm': 'auto'
    },
    'XGB Classifier': {
        'max_depth': 3,
        'learning_rate': 0.1,
        'n_estimators': 100,
        'booster': 'gbtree',
        'gamma': 0,
        'use_label_encoder':False, 
        'eval_metric':'logloss'
    }
}

Configuré el modelo y los hiperparámetros que usaríamos en el diccionario separado. Experimentaríamos con cada modelo y el hiperparámetro para ver si MLFlow podía rastrearlos.

El último paso es ejecutar el experimento y dejar que MLFlow los rastree. Para hacer eso, podemos usar el siguiente código.

In [ ]:
from mlflow.models import infer_signature

In [ ]:
for model_name in models_to_experiment.keys():
    RUN_NAME = f'Titanic Classifier Experiment {model_name}'
    with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME):
 
        params = params_to_experiment[model_name]
        model = models_to_experiment[model_name](**params)
        
        model.fit(X_train, y_train)  # Train model
        predictions = model.predict(X_test)  # Predictions
    
        # Calculate metrics
        accuracy = accuracy_score(y_test, predictions)
        f1 = f1_score(y_test, predictions, average='weighted')  
        
        # Log the hyperparameters
        mlflow.log_params(params)
    
        # Log the loss metric
        mlflow.log_metric(f"{model_name}_accuracy", accuracy)
        mlflow.log_metric(f"{model_name}_f1", f1)
    
        # Set a tag that we can use to remind ourselves what this run was for
        mlflow.set_tag("Training Info", f"{model_name} model for Titanic")
    
        # Infer the model signature
        signature = infer_signature(X_train, model.predict(X_train))

        #log the model
        if model_name == "XGBoost":
            model_info = mlflow.xgboost.log_model(
            xgb_model=model,
            artifact_path=f"titanic_{model_name}_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"tracking-titanic-{model_name}",
        )
        else:
            model_info = mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path=f"titanic_{model_name}_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"tracking-titanic-{model_name}",
        )

        #Log the dataset
        training_df = pd.concat([X_train, y_train], axis =1).reset_index(drop = True)
        training_df["ModelOutput"] = model.predict(X_train)
        dataset = mlflow.data.from_pandas(training_df, targets="survived", predictions="ModelOutput", name = f"data_{model_name}")
        mlflow.log_input(dataset, context="training data")

        mlflow.end_run() 

El registra el modelo para que pueda usarse para predecir

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()

client.set_registered_model_tag("tracking-titanic-LR", "task", "classification")

todo lo que registre durante el proceso anterior se almacenará en MLFlow.

    RUN_NAME = f'Titanic Classifier Experiment {model_name}'
    with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME)

El parámetro RUN_NAME controlaría el nombre de ejecución que se muestra en la interfaz de usuario. Elija un nombre apropiado que pueda recordar. Luego, hay algunas funciones que utilizamos para realizar un seguimiento:

mlflow.log_params: Realice un seguimiento de los hiperparámetros del modelo con el que experimentamos,

mlflow.log_metric: Realice un seguimiento de las métricas del modelo,

mlflow.set_tag: Entrada para proporcionar una etiqueta para la información de ejecución,

infer_signature: Inferir una firma de modelo de MLflow, que es la información de entrada y salida de datos,

mlflow.xgboost.log_model y mlflow.sklearn.log_model: Realizar un seguimiento de la información y el artefacto del modelo. Consulte esta lista para conocer la lista de marcos integrados que podemos rastrear con MLFlow.

mlflow.log_input: realice un seguimiento del conjunto de datos utilizado para la capacitación.

# Parte 2

En este ejemplo, entrenaríamos un modelo de árbol de decisión y lo registraríamos.

La versión 1 del modelo es el primer modelo que registramos con ese nombre. Si registramos otro modelo con el mismo nombre, se podría actualizar a otra versión.

Seleccionar la Versión le permite ver los detalles del modelo mucho mejor.

In [ ]:
model_name = 'Decision Tree'
RUN_NAME = f'Titanic Classifier Experiment {model_name}'
params = {'max_depth':3, 'min_samples_split':2}
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME):
    
    model = decision_tree = DecisionTreeClassifier(**params)
    
    model.fit(X_train, y_train)  # Train model
    predictions = model.predict(X_test)  # Predictions

    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')  
    
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric(f"{model_name}_accuracy", accuracy)
    mlflow.log_metric(f"{model_name}_f1", f1)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", f"{model_name} model for Titanic")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))
    
    #log the model

    model_info = mlflow.sklearn.log_model(
    sk_model=model,
    artifact_path=f"titanic_{model_name}_model",
    signature=signature,
    input_example=X_train,
    registered_model_name=f"tracking-titanic-{model_name}",)

    mlflow.end_run() 

## Etiquetas y alias

Las etiquetas son útiles cuando tienes varios modelos registrados y solo deseas obtener un modelo específico. Utilice las etiquetas de una manera que usted o el equipo puedan entender.

Aca se registra el modelo para generar prediciones

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()

client.set_registered_model_tag("tracking-titanic-Decision Tree", "task", "classification")

También puede cambiar las etiquetas de versión del modelo con el siguiente código.

In [ ]:
client.set_model_version_tag("tracking-titanic-Decision Tree", "1", "validation_status", "approved")

También es posible darle un Alias ​​al modelo. Por ejemplo, damos al modelo Versión 1 como “campeón”.

In [ ]:
client.set_registered_model_alias("tracking-titanic-Decision Tree", "champion", "1")

Además, puede eliminar las etiquetas y el alias con el siguiente código.

In [ ]:
# Delete registered model tag
client.delete_registered_model_tag("tracking-titanic-Decision Tree", "task")

# Delete model version tag
client.delete_model_version_tag("tracking-titanic-Decision Tree", "1", "validation_status")

# delete the alias
client.delete_registered_model_alias("tracking-titanic-Decision Tree", "Champion")

Por último, puede agregar la descripción del modelo para cada versión con el siguiente código.

In [ ]:
client.update_model_version(
    name="tracking-titanic-Decision Tree",
    version="1",
    description="This model is a sample titanic model with decision tree as baseline",
)

O desea actualizar toda la descripción del modelo, puede utilizar el siguiente código.

In [ ]:
client.update_registered_model(
    name="tracking-titanic-Decision Tree",
    description="Titanic Decision Tree Model",
)

## Obtención de modelos

El Registro de Modelos también podría funcionar como recuperación de modelos. Siempre podremos acceder al modelo que tenemos registrado y crear predicciones a partir de ese modelo.

Por ejemplo, buscaríamos el modelo que desarrollamos anteriormente.

In [ ]:
import mlflow.pyfunc

In [ ]:
model_name = "tracking-titanic-Decision Tree"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

Luego, predeciríamos nuestro conjunto de datos con el siguiente código.

In [ ]:
model.predict(X_test)

¿Qué tan fácil es eso? También es posible recuperar el modelo con el Alias.

In [ ]:
alias = 'champion'
champion_version = mlflow.pyfunc.load_model(f"models:/{model_name}@{alias}")

champion_version.predict(X_test)

In [ ]:
print('ok_')